In [190]:
import numpy as np
import re

In [73]:
def read_input(infile):
    data = []
    with open(infile, 'r') as inf:
        for line in inf.readlines():
            if line.strip() == '':
                continue
            data.append([c for c in line.strip()])
        
    # print(data)
    return np.array(data)

In [272]:
def find_longest_path(data, starting_char):
    visited = np.zeros(data.shape, dtype=int)
    dist = np.ones(data.shape, dtype=int) + 10000000
    
    neighbours = {'|': ((-1, 0), (1, 0)),
                  '-': ((0, -1), (0, 1)),
                  'L': ((-1, 0), (0, 1)),
                  'J': ((-1, 0), (0, -1)),
                  '7': ((1, 0), (0, -1)),
                  'F': ((1, 0), (0, 1))}
    
    y, x = np.where(data == 'S')
    y = y[0]
    x = x[0]
    data[y, x] = starting_char
    visited[y, x] = 1
    dist[y, x] = 0
    next_pos = [(y+ny, x+nx) for ny, nx in neighbours[data[y, x][0]]]
    
    for ny, nx in next_pos:
        dist[ny, nx] = 1
    
    done = False
    while not done:
        go_next = []
        for y, x in next_pos:
            visited[y, x] = 1
            for ny, nx in neighbours[data[y, x]]:
                ny += y
                nx += x
                if not visited[ny, nx] or dist[ny, nx] > dist[y, x]+1:
                    # Go there
                    go_next.append((ny, nx))
                    dist[ny, nx] = dist[y, x] + 1
                
            next_pos = go_next
                
        if len(go_next) == 0:
            done = True
            
    return(np.max(dist[np.where(dist<10000000)])), visited
        

In [273]:
def count_pipes_x(d):
    s = np.sum(d == '|')
    st = ''.join(d)
    s += len(re.findall('F-*J', st))
    s += len(re.findall('L-*7', st))

    return s

def count_pipes_y(d):
    s = np.sum(d == '-')
    st = ''.join(d)
    s += len(re.findall('F\|*J', st))
    s += len(re.findall('7\|*L', st))
           
    return s


In [274]:
def count_enclosed(pipemap, data):
    enclosed = np.pad(pipemap, ((1, 1), (1, 1)), 'constant', constant_values=((0, 0), (0, 0)))
    
    large_data = np.pad(data, ((1, 1), (1, 1)), 'constant', constant_values=((0, 0), (0, 0)))
    large_data[np.where(enclosed != 1)] = '.'
    
    y, x  = 0, 0
    
    nextpos = [(1, 0), (0, 1)]

    done = False
    while not done:
        go_next = []
        for next_y, next_x in nextpos:
            for dy, dx in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                if enclosed[next_y+dy, next_x+dx] == 0:
                    # Reachable:
                    enclosed[next_y+dy, next_x+dx] = 1
                    go_next.append((next_y+dy, next_x+dx))
        nextpos = go_next
        if len(nextpos) == 0:
            done = True
            
    it = np.nditer(enclosed, flags=['multi_index'])
    
    for p in it:
        if p == 0:
            y, x = it.multi_index
            
            if count_pipes_x(large_data[y, :x]) % 2 == 0:
                enclosed[y, x] = 1
                continue
            if count_pipes_x(large_data[y, x+1:]) % 2 == 0:
                enclosed[y, x] = 1
                continue
            if count_pipes_y(large_data[:y, x]) % 2 == 0:
                enclosed[y, x] = 1
                continue
            if count_pipes_y(large_data[y+1:, x]) % 2 == 0:
                enclosed[y, x] = 1
                continue

    return len(enclosed[np.where(enclosed==0)])  
    

In [276]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

data = read_input('input10a.txt')

maxdist, _ = find_longest_path(data, 'F')

print(f'Max dist is {maxdist}')

assert maxdist == 8

print('\nPuzzle case\n')

data = read_input('input10.txt')

maxdist, _ = find_longest_path(data, 'L')

print(f'Max dist is {maxdist}')

assert maxdist == 6890

print('\n*****\nPuzzle2\n*****\n')

print('Test case\n')

data = read_input('input10b.txt')

_, visited = find_longest_path(data, 'F')

n = count_enclosed(visited, data)

print(f'N enclosed is {n}')

assert n == 4

print('Test case 2\n')

data = read_input('input10c.txt')

_, visited = find_longest_path(data, 'F')

n = count_enclosed(visited, data)

print(f'N enclosed is {n}')

assert n == 8

print('Test case 3\n')

data = read_input('input10d.txt')

_, visited = find_longest_path(data, 'F')

n = count_enclosed(visited, data)

print(f'N enclosed is {n}')

assert n == 10

print('\nPuzzle case\n')

data = read_input('input10.txt')

_, visited = find_longest_path(data, 'L')

n = count_enclosed(visited, data)

print(f'N enclosed is {n}')

assert n == 453


*****
Puzzle1
*****

Test case

Max dist is 8

Puzzle case

Max dist is 6890

*****
Puzzle2
*****

Test case

N enclosed is 4
Test case 2

N enclosed is 8
Test case 3

N enclosed is 10

Puzzle case

N enclosed is 453
